In [1]:
import numpy as np

import fiftyone as fo
import fiftyone.zoo as foz

/home/sashi/miniconda3/envs/anime-data-process/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/sashi/miniconda3/envs/anime-data-process/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
dataset_dir = "/media/sashi/DATA1/pictures/ACG-extra/training_data/yama_no_susume/screenshots/S4/EP12/"
dataset = fo.Dataset.from_dir(dataset_dir, dataset_type=fo.types.ImageDirectory)

model = foz.load_zoo_model("mobilenet-v2-imagenet-torch")
embeddings = dataset.compute_embeddings(model)

print(embeddings.shape)

 100% |███████████████| 6008/6008 [587.0ms elapsed, 0s remaining, 10.2K samples/s]      


/home/sashi/miniconda3/envs/anime-data-process/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/sashi/miniconda3/envs/anime-data-process/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


 100% |███████████████| 6008/6008 [4.1m elapsed, 0s remaining, 25.3 samples/s]      
(6008, 1280)


In [3]:
from tqdm import tqdm

def mark_duplicate(subdataset, similarity_matrix, thresh=0.985):
    
    n = len(similarity_matrix)
    similarity_matrix = similarity_matrix - np.identity(n)
    
    id_map = [s.id for s in subdataset.select_fields(["id"])]
    samples_to_remove = set()
    samples_to_keep = set()

    for idx, sample in enumerate(subdataset):
        max_similarity = similarity_matrix[idx].max()
        sample["max_similarity"] = max_similarity
        sample.save()

    for idx, sample in tqdm(enumerate(subdataset)):
        if sample.id not in samples_to_remove:
            # Keep the first instance of two duplicates
            samples_to_keep.add(sample.id)

            dup_idxs = np.where(similarity_matrix[idx] > thresh)[0]
            for dup in dup_idxs:
                # We kept the first instance so remove all other duplicates
                samples_to_remove.add(id_map[dup])

            if len(dup_idxs) > 0:
                sample.tags.append("has_duplicates")
                sample.save()

        else:
            sample.tags.append("duplicate")
            sample.save()
    return samples_to_remove, samples_to_keep

In [4]:
from sklearn.metrics.pairwise import cosine_similarity

max_compare_size = 10000
thresh = 0.985

samples_to_remove = set()
samples_to_keep = set()

for k in range(0, len(embeddings), max_compare_size):
    end = min(k + max_compare_size, len(embeddings))
    similarity_matrix = cosine_similarity(embeddings[k:end])
    samples_to_remove_sub, samples_to_keep_sub = mark_duplicate(
        dataset[k:end], similarity_matrix, thresh)
    samples_to_remove = samples_to_remove | samples_to_remove_sub
    samples_to_keep = samples_to_keep | samples_to_keep_sub

6008it [00:02, 2072.36it/s]


In [5]:
session = fo.launch_app(dataset)

/home/sashi/miniconda3/envs/anime-data-process/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
import os
for sample_id in tqdm(samples_to_remove):
    os.remove(dataset[sample_id].filepath)
dataset.delete_samples(list(samples_to_remove))

100%|█████████████████████████████████████████████████████████| 5281/5281 [00:03<00:00, 1752.00it/s]
